In [1]:
import os, sys
sys.path.append(os.path.abspath(".."))

import torch
from nanochat.gpt import GPT, GPTConfig

# 1. 定义配置 (这里使用一个小模型作为演示)
config = GPTConfig(
    vocab_size=50304, # 词表大小
    sequence_len=1024,# 最大上下文长度
    n_layer=4,        # Transformer 层数
    n_head=4,         # 注意力头数
    n_kv_head=4,   
    n_embd=128        # 嵌入维度
)

# 2. 实例化模型
model = GPT(config)

# 3. 初始化权重 (关键步骤)
# 应用特定的正态分布初始化，并将 lm_head 和部分投影层置零
model.init_weights()

print(f"模型参数量: {sum(p.numel() for p in model.parameters())}")

# --- 场景 A: 推理 (Inference) ---
# 构造模拟输入: Batch Size=2, Sequence Length=16
B, T = 2, 5
idx = torch.randint(0, config.vocab_size, (B, T))

# 前向传播 (不传 targets)
logits = model(idx)

# 输出形状: (Batch, Seq_Len, Vocab_Size)
print(f"\n输入形状: {idx.shape}")
print(f"Logits 形状: {logits.shape}") # -> torch.Size([2, 16, 50304])


# --- 场景 B: 训练 (Training) ---
# 构造模拟目标 (通常是 idx 向后移一位)
targets = torch.randint(0, config.vocab_size, (B, T))

# 前向传播 (传入 targets)
loss = model(idx, targets=targets)

# 输出是一个标量 Loss
print(f"\nTraining Loss: {loss.item():.4f}")

模型参数量: 13664256

输入形状: torch.Size([2, 5])
Logits 形状: torch.Size([2, 5, 50304])

Training Loss: 10.8258


In [2]:
idx

tensor([[50225,  8774,  1447,  6861, 46644, 25897, 39148, 21490, 15152, 29623,
          1377, 18267, 20870, 17969, 33869, 40159],
        [41792, 36984, 17243, 16975, 12120, 11724, 27727, 35220, 29049, 11822,
          5502, 20497, 14346, 23403, 19113, 21961]])

In [ ]:
next_token = logits[:, -1, :]
print(f"next_token形状: {next_token.shape}")

In [1]:
import os, sys
sys.path.append(os.path.abspath(".."))

from nanochat.tokenizer import get_tokenizer

text = "Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?"

tok = get_tokenizer()

# 1) 仅对这段 user 文本做 tokenize
ids = tok.encode(text)  # list[int]
tokens = [tok.id_to_token(i) for i in ids]  # list[str], 每个元素就是一个 token 的“字符串形态”
print("NUM_TOKENS:", len(tokens))
print(tokens)  # 如果太长你也可以逐行 print

# 2) 你的整条 GSM8K conversation 的“完整 token 流”
conv = {
  "messages": [
    {"role": "user", "content": text},
    {"role": "assistant", "content": [
      {"type": "text", "text": "Weng earns 12/60 = "},
      {"type": "python", "text": "12/60"},
      {"type": "python_output", "text": "0.2"},
      {"type": "text", "text": " per minute. Working 50 minutes, she earned 0.2*50 = "},
      {"type": "python", "text": "0.2*50"},
      {"type": "python_output", "text": "10"},
      {"type": "text", "text": ".\n#### 10"},
    ]},
  ]
}

full_ids, mask = tok.render_conversation(conv)
full_tokens = [tok.id_to_token(i) for i in full_ids]
print("FULL_NUM_TOKENS:", len(full_tokens))
print(full_tokens)

# 可选：同时打印 (token, id, mask) 方便核对哪些 token 来自 user/assistant/tool output
triples = [(repr(t), i, m) for t, i, m in zip(full_tokens, full_ids, mask)]
print(triples)

NUM_TOKENS: 33
['W', 'eng', ' earns', ' $', '12', ' an', ' hour', ' for', ' bab', 'ys', 'itting', '.', ' Y', 'ester', 'day', ',', ' she', ' just', ' did', ' ', '50', ' minutes', ' of', ' bab', 'ys', 'itting', '.', ' How', ' much', ' did', ' she', ' earn', '?']
FULL_NUM_TOKENS: 92
['<|bos|>', '<|user_start|>', 'W', 'eng', ' earns', ' $', '12', ' an', ' hour', ' for', ' bab', 'ys', 'itting', '.', ' Y', 'ester', 'day', ',', ' she', ' just', ' did', ' ', '50', ' minutes', ' of', ' bab', 'ys', 'itting', '.', ' How', ' much', ' did', ' she', ' earn', '?', '<|user_end|>', '<|assistant_start|>', 'W', 'eng', ' earns', ' ', '12', '/', '60', ' =', ' ', '<|python_start|>', '12', '/', '60', '<|python_end|>', '<|output_start|>', '0', '.', '2', '<|output_end|>', ' per', ' minute', '.', ' Working', ' ', '50', ' minutes', ',', ' she', ' earned', ' ', '0', '.', '2', '*', '50', ' =', ' ', '<|python_start|>', '0', '.', '2', '*', '50', '<|python_end|>', '<|output_start|>', '10', '<|output_end|>', '.\n', '#

In [2]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from tasks.smoltalk import SmolTalk

def main():
    print("正在加载 SmolTalk 数据集 (split='train')...")
    
    # 1. 初始化任务
    # split 参数必须是 "train" 或 "test"
    # 第一次运行时，datasets 库会自动下载数据并缓存
    task = SmolTalk(split="train")
    
    print(f"数据集加载完成，总样本数: {len(task)}")
    
    # 2. 获取并展示第一个样本
    print("\n" + "="*50)
    print("查看第 1 个样本 (Index 0):")
    print("="*50)
    
    example = task[0]
    messages = example["messages"]
    
    for i, msg in enumerate(messages):
        role = msg["role"].upper()
        content = msg["content"]
        # 为了展示整洁，过长的内容进行截断
        preview = content[:80].replace('\n', ' ') + "..." if len(content) > 80 else content.replace('\n', ' ')
        print(f"Message {i} [{role}]: {preview}")

    # 3. 展示切片功能 (使用 Task 基类提供的功能)
    # 例如：只获取第 10 到第 12 个样本
    print("\n" + "="*50)
    print("查看切片样本 (Index 10-12):")
    print("="*50)
    
    sliced_task = SmolTalk(split="train", start=10, stop=12)
    
    for i in range(len(sliced_task)):
        # 注意：sliced_task[0] 实际上对应原始数据集的 task[10]
        ex = sliced_task[i]
        first_user_msg = ex["messages"][0]
        # 如果第一条是 system，取第二条
        if first_user_msg["role"] == "system":
             first_user_msg = ex["messages"][1]
             
        print(f"Slice Item {i} (Original Index {10+i}): User asks -> {first_user_msg['content'][:60]}...")

if __name__ == "__main__":
    main()

/Users/a/Desktop/xinao/reborn1000/build_sell_invest/nanochat/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


正在加载 SmolTalk 数据集 (split='train')...
数据集加载完成，总样本数: 460341

查看第 1 个样本 (Index 0):
Message 0 [USER]: A researcher interested in examining students' study time per day collected data...
Message 1 [ASSISTANT]: To calculate the mean, we need to add up all the values and divide by the total ...
Message 2 [USER]: How would you describe the dispersion of this data set. Which measurement or cal...
Message 3 [ASSISTANT]: To describe the dispersion of the data, I would calculate the standard deviation...
Message 4 [USER]: Would a median and or a mode be a better representation for the central tendency...
Message 5 [ASSISTANT]: In this dataset, the median and mode might be more representative of the central...

查看切片样本 (Index 10-12):
Slice Item 0 (Original Index 10): User asks -> Your response should contain at least 3 sentences. Include k...
Slice Item 1 (Original Index 11): User asks -> Write a Python function to return a dictionary value by a gi...


In [4]:
task = SmolTalk(split="train")
    
print(f"数据集加载完成，总样本数: {len(task)}")

# 2. 获取并展示第一个样本
print("\n" + "="*50)
print("查看第 1 个样本 (Index 0):")
print("="*50)

example = task[0]
messages = example["messages"]

数据集加载完成，总样本数: 460341

查看第 1 个样本 (Index 0):


In [5]:
messages

[{'content': "A researcher interested in examining students' study time per day collected data from a random sample of 20 students. Here are the data: \n\n3, 1, 2, 4, 5, 2, 3, 4, 2, 1, 2, 3, 4, 6, 3, 2, 2, 1, 5, 4\n\nWhat is the mean of this dataset? How would you describe or interpret the result?",
  'role': 'user'},
 {'content': 'To calculate the mean, we need to add up all the values and divide by the total number of observations. \n\nThe sum of the values is 3 + 1 + 2 + 4 + 5 + 2 + 3 + 4 + 2 + 1 + 2 + 3 + 4 + 6 + 3 + 2 + 2 + 1 + 5 + 4 = 59.\n\nThere are 20 observations in the dataset. The mean is calculated by dividing the sum by the number of observations: 59 / 20 = 2.95.\n\nThe mean of 2.95 hours per day indicates that, on average, students in this sample spend approximately 3 hours per day studying. This result suggests a relatively moderate amount of daily study time, with most students likely dedicating a few hours to studying each day. However, to better understand the study 

In [7]:
task[1]

{'messages': [{'content': "You're an AI assistant for text re-writing. Rewrite the input text to make it more professional and formal while retaining its essential content.",
   'role': 'system'},
  {'content': "Hey Alex,\n\nSo, I've been thinking a lot about our new project on the etymology of ancient languages, and I have a ton of ideas! I was reading this fascinating article about the Proto-Indo-European roots of some modern words, and it got me thinking about how we could really dive deep into this. Like, we could look at how certain words have evolved over time and see if there are any patterns or commonalities. \n\nI know it's a lot to take in, but I think it could be really interesting. Let me know what you think, and maybe we can set up a time to discuss it further. Oh, and don't forget about the conference next month! 😊\n\nTalk soon!\nEleanor",
   'role': 'user'},
  {'content': 'Dear Alex,\n\nI hope this message finds you well. I have been giving considerable thought to our up

In [8]:
task.num_examples()

460341

In [9]:
task.get_example(0)

{'messages': [{'content': "A researcher interested in examining students' study time per day collected data from a random sample of 20 students. Here are the data: \n\n3, 1, 2, 4, 5, 2, 3, 4, 2, 1, 2, 3, 4, 6, 3, 2, 2, 1, 5, 4\n\nWhat is the mean of this dataset? How would you describe or interpret the result?",
   'role': 'user'},
  {'content': 'To calculate the mean, we need to add up all the values and divide by the total number of observations. \n\nThe sum of the values is 3 + 1 + 2 + 4 + 5 + 2 + 3 + 4 + 2 + 1 + 2 + 3 + 4 + 6 + 3 + 2 + 2 + 1 + 5 + 4 = 59.\n\nThere are 20 observations in the dataset. The mean is calculated by dividing the sum by the number of observations: 59 / 20 = 2.95.\n\nThe mean of 2.95 hours per day indicates that, on average, students in this sample spend approximately 3 hours per day studying. This result suggests a relatively moderate amount of daily study time, with most students likely dedicating a few hours to studying each day. However, to better unders

In [10]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from tasks.common import TaskMixture, Task
from tasks.smoltalk import SmolTalk
from tasks.mmlu import MMLU
from tasks.gsm8k import GSM8K

# 方式1: 使用现有的 Task 类创建 TaskMixture
train_dataset = TaskMixture([
    SmolTalk(split="train"),  # 460K 条对话
    MMLU(subset="auxiliary_train", split="train"),  # 100K 条多选题
    GSM8K(subset="main", split="train"),  # 8K 条数学题
])

# 直接通过索引访问数据
print(f"数据集总长度: {len(train_dataset)}")
print(f"\n访问第 0 条数据:")
conversation_0 = train_dataset[0]
print(conversation_0)

print(f"\n访问第 100 条数据:")
conversation_100 = train_dataset[100]
print(conversation_100)

print(f"\n访问最后一条数据:")
conversation_last = train_dataset[len(train_dataset) - 1]
print(conversation_last)

# 遍历前几条数据
print(f"\n遍历前 5 条数据:")
for i in range(min(5, len(train_dataset))):
    conv = train_dataset[i]
    print(f"索引 {i}: 包含 {len(conv.get('messages', []))} 条消息")

数据集总长度: 567656

访问第 0 条数据:
{'messages': [{'role': 'user', 'content': "Multiple Choice question: Hadrian's Wall Hadrian's Wall was built, beginning in 122 AD, to keep Roman Britain safe from attacks from the Picts. The wall, starting from the North Sea to the Irish Sea (from the Tyne to the Solway), is about 73 miles long, 10 feet wide, and 15 feet high. In addition to the wall, the Romans built a system of small forts  housing up to 60 men every Roman mile along its whole length, with e towers very 1/3 Roman mile. 16 larger forts holding from 500 to 1000 soldiers were built into the wall, with large gates on the north face. Israeli West Bank barrier The Israeli West Bank barrier is a barrier under construction by the State of Israel along and within the West Bank. Upon completion, the barrier's total length will be nearly 700 kilometers. The barrier is built mainly in the West Bank. The Berlin Wall The Berlin Wall was the physical boundary between West Germany and East Germany. The Ber

In [11]:
train_dataset[0]

{'messages': [{'role': 'user',
   'content': "Multiple Choice question: Hadrian's Wall Hadrian's Wall was built, beginning in 122 AD, to keep Roman Britain safe from attacks from the Picts. The wall, starting from the North Sea to the Irish Sea (from the Tyne to the Solway), is about 73 miles long, 10 feet wide, and 15 feet high. In addition to the wall, the Romans built a system of small forts  housing up to 60 men every Roman mile along its whole length, with e towers very 1/3 Roman mile. 16 larger forts holding from 500 to 1000 soldiers were built into the wall, with large gates on the north face. Israeli West Bank barrier The Israeli West Bank barrier is a barrier under construction by the State of Israel along and within the West Bank. Upon completion, the barrier's total length will be nearly 700 kilometers. The barrier is built mainly in the West Bank. The Berlin Wall The Berlin Wall was the physical boundary between West Germany and East Germany. The Berlin Wall was built in th